In [ ]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/')
sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/dl')
import dl.loaders.ultrasound_dataset as ultrasound_dataset
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.loaders.mr_us_dataset as mr_us_dataset
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src/')




In [2]:
mount_point = '/mnt/raid/C1_ML_Analysis'

In [ ]:

df = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/C_dataset_analysis_protocoltagsonly_gaboe230_masked_resampled_256_spc075.csv')
df_study = df.query('study_id == "UNC-1037-5"')
print(df_study)

In [4]:
from generative.networks.nets.autoencoderkl import Encoder

latent_dim = 3

    

In [ ]:
vs = us_simu.VolumeSamplingBlindSweep(mount_point=mount_point).cuda()

In [ ]:

model = us_simu.USPCReconstruction(mount_point=mount_point, lr=1e-4, weight_decay=0.01, latent_channels=3, num_samples=4096, input_dim=6, output_dim=3, K=[27, 27], num_heads=[64, 128], stages=[64, 128], dropout=0.1, pooling_factor=[0.5, 0.5], score_pooling=0, pooling_hidden_dim=[32, 64], feed_forward_hidden_dim=[32, 64], use_skip_connection=0)
model.cuda()

In [ ]:
df_train = pd.read_csv(os.path.join(mount_point, 'CSV_files/C1_C2_cines_gt_ga_withmeta_masked_resampled_256_spc075_uuid_study_train_protocoltags.csv'))
df_val = pd.read_csv(os.path.join(mount_point, 'CSV_files/C1_C2_cines_gt_ga_withmeta_masked_resampled_256_spc075_uuid_study_valid_protocoltags.csv'))
df_test = pd.read_csv(os.path.join(mount_point, 'CSV_files/C1_C2_cines_gt_ga_withmeta_masked_resampled_256_spc075_uuid_study_test_protocoltags.csv'))

dm = ultrasound_dataset.USDataModuleBlindSweepWTag(df_train, df_val, df_test, mount_point=mount_point, batch_size=2, num_frames=0, img_column='file_path', ga_column='ga_boe', id_column='study_id', max_sweeps=3)
dm.setup()
# ds = ultrasound_dataset.USDatasetBlindSweepWTag(df_train, mount_point=mount_point, num_frames=0, img_column='file_path', ga_column='ga_boe', transform=None, id_column='study_id', max_sweeps=3)



In [ ]:
b0 = next(iter(dm.train_dataloader()))
b0[0].shape

In [ ]:
b0['tag'][:,0].shape